[View in Colaboratory](https://colab.research.google.com/github/vivek6449/Building_Damage_Classification/blob/master/Building_train_set.ipynb)

In [1]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
from google.colab import auth 
auth.authenticate_user() 

In [0]:
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 

In [5]:
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive 

In [7]:
print ('Files in Drive:')  
!ls drive/ 

Files in Drive:
Colab Notebooks      LSTM			Study
Docs		     Presentation.pdf		Untitled presentation.pdf
Job Description.odt  Resumes
Job Tracker.ods      Rotor Crack Detection.odt


In [8]:
!pip install lightgbm

    100% |████████████████████████████████| 737kB 6.6MB/s 


In [0]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMClassifier

import gc

## Importing the data

In [0]:
buil_owner = pd.read_csv('drive/Colab Notebooks/Building/Building_Ownership_Use.csv')
buil_stru = pd.read_csv('drive/Colab Notebooks/Building/Building_Structure.csv')
train = pd.read_csv('drive/Colab Notebooks/Building/train.csv')
test = pd.read_csv('drive/Colab Notebooks/Building/test.csv')

Dropping 'building_id' from `Building_Structure` file so we can concatenate `Building_Sturcture` and `Building_Ownership_Use` files.

In [0]:
buil_stru.drop(buil_stru.columns[0], axis=1, inplace=True)

Combine both DataFrames and make new one called `combined_new`.

In [0]:
combined_new = pd.concat([buil_owner, buil_stru], axis=1)

We don't need `Building_Structure` and `Building_Ownership_Use` dataframe anymore. So we delete it both to clear some memory.

In [13]:
del buil_stru, buil_owner
gc.collect()

11

Let's check Object values and Null values in out DataFrame. 

* First we will deal with objects so we can convert it into Numerical values using dummies.

In [14]:
combined_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052948 entries, 0 to 1052947
Data columns (total 45 columns):
building_id                               1052948 non-null object
district_id                               1052948 non-null int64
vdcmun_id                                 1052948 non-null int64
ward_id                                   1052948 non-null int64
legal_ownership_status                    1052948 non-null object
count_families                            1052946 non-null float64
has_secondary_use                         1052938 non-null float64
has_secondary_use_agriculture             1052948 non-null int64
has_secondary_use_hotel                   1052948 non-null int64
has_secondary_use_rental                  1052948 non-null int64
has_secondary_use_institution             1052948 non-null int64
has_secondary_use_school                  1052948 non-null int64
has_secondary_use_industry                1052948 non-null int64
has_secondary_use_health_post       

With the use of `One_Hot_Encoding`, we will convert all the object values into Numerical features.

But first check all the `object` featues to get an idea how many classes are belongs to individual feature.

In [15]:
combined_new.land_surface_condition.value_counts()

Flat              874202
Moderate slope    144748
Steep slope        33998
Name: land_surface_condition, dtype: int64

In [16]:
combined_new.foundation_type.value_counts()

Mud mortar-Stone/Brick    886331
Bamboo/Timber              60598
Cement-Stone/Brick         57354
RC                         42808
Other                       5857
Name: foundation_type, dtype: int64

In [17]:
combined_new.roof_type.value_counts()

Bamboo/Timber-Light roof    739184
Bamboo/Timber-Heavy roof    248235
RCC/RB/RBC                   65529
Name: roof_type, dtype: int64

In [18]:
combined_new.ground_floor_type.value_counts()

Mud            847250
Brick/Stone    100487
RC              99046
Timber           4072
Other            2093
Name: ground_floor_type, dtype: int64

In [19]:
combined_new.other_floor_type.value_counts()

TImber/Bamboo-Mud    666182
Timber-Planck        177571
Not applicable       160301
RCC/RB/RBC            48894
Name: other_floor_type, dtype: int64

In [20]:
combined_new.position.value_counts()

Not attached       816364
Attached-1 side    173459
Attached-2 side     53522
Attached-3 side      9602
Name: position, dtype: int64

In [21]:
combined_new.plan_configuration.value_counts()

Rectangular                        1009987
Square                               23106
L-shape                              15088
Multi-projected                       1412
T-shape                               1302
Others                                 994
U-shape                                591
Building with Central Courtyard        203
E-shape                                167
H-shape                                 97
Name: plan_configuration, dtype: int64

In [22]:
combined_new.condition_post_eq.value_counts()

Damaged-Not used                           249464
Damaged-Repaired and used                  211119
Damaged-Used in risk                       187480
Damaged-Rubble unclear                     145732
Damaged-Rubble clear                       132508
Not damaged                                 71587
Damaged-Rubble Clear-New building built     54587
Covered by landslide                          471
Name: condition_post_eq, dtype: int64

In [23]:
combined_new.legal_ownership_status.value_counts()

Private          1014042
Public             22326
Institutional      10502
Other               6078
Name: legal_ownership_status, dtype: int64

I have used pandas command `pd.get_dummies()` to numerical vector or each class of features.

In [0]:
land_dum = pd.get_dummies(combined_new.land_surface_condition)
foun = pd.get_dummies(combined_new.foundation_type)
roof = pd.get_dummies(combined_new.roof_type)
grnd = pd.get_dummies(combined_new.ground_floor_type)
oth = pd.get_dummies(combined_new.other_floor_type)
pos = pd.get_dummies(combined_new.position)
plan = pd.get_dummies(combined_new.plan_configuration)
con = pd.get_dummies(combined_new.condition_post_eq)
legal = pd.get_dummies(combined_new.legal_ownership_status)

In [0]:
cols = [land_dum, foun, roof, grnd, oth, pos, plan, con, legal]

In [0]:
dummy_cols = pd.concat(cols, axis=1)

In [27]:
del land_dum, foun, roof, grnd, oth, pos, plan, con, legal
gc.collect()

0

In [0]:
building_id = combined_new.building_id

In [0]:
drop_cols = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration',
            'condition_post_eq', 'legal_ownership_status','district_id', 'vdcmun_id','building_id','ward_id']

In [0]:
combined_new.drop(drop_cols, axis=1, inplace=True)

In [0]:
combined_new = pd.concat([combined_new, dummy_cols], axis=1)

Converting all the datatype to `float`.

In [0]:
combined_new = combined_new.astype(float)

In [0]:
# add building id to our dataframe
combined_new = pd.concat([building_id, combined_new], axis=1)
combined_new.set_index('building_id',inplace=True)

Our train dataframe's shape is 631761 x 75 (631761 rows and 75 columns). To get same `building_id` as index in our train set we need to set our `combined_new` dataframe same as our `train` dataframe index. If we apply for loop to whole dataframe , it would be more time consuming. so I have divided entire dataframe in 50000 size of chunks and later we will concat it on rows.

In [0]:
train_df = pd.DataFrame([])
for i in range(50000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df = train_df.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

0 % records
20 % records
40 % records
60 % records
80 % records


In [0]:
train_df_1 = pd.DataFrame([])
for i in range(50000,100000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_1 = train_df_1.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

100 % records
120 % records
140 % records
160 % records
180 % records


In [0]:
train_df_2 = pd.DataFrame([])
for i in range(100000,150000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_2 = train_df_2.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

200 % records
220 % records
240 % records
260 % records
280 % records


In [0]:
train_df_3 = pd.DataFrame([])
for i in range(150000,200000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_3 = train_df_3.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

300 % records
320 % records
340 % records
360 % records
380 % records


In [0]:
train_df_4 = pd.DataFrame([])
for i in range(200000,250000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_4 = train_df_4.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

400 % records
420 % records
440 % records
460 % records
480 % records


In [0]:
train_df_5 = pd.DataFrame([])
for i in range(250000,300000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_5 = train_df_5.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

500 % records
520 % records
540 % records
560 % records
580 % records


In [0]:
train_df_6 = pd.DataFrame([])
for i in range(300000,350000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_6 = train_df_6.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

600 % records
620 % records
640 % records
660 % records
680 % records


In [0]:
train_df_7 = pd.DataFrame([])
for i in range(350000,400000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_7 = train_df_7.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

700 % records
720 % records
740 % records
760 % records
780 % records


In [0]:
train_df_8 = pd.DataFrame([])
for i in range(400000,450000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_8 = train_df_8.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

800 % records
820 % records
840 % records
860 % records
880 % records


In [0]:
train_df_9 = pd.DataFrame([])
for i in range(450000,500000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_9 = train_df_9.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

900 % records
920 % records
940 % records
960 % records
980 % records


In [0]:
train_df_10 = pd.DataFrame([])
for i in range(500000,550000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_10 = train_df_10.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

1000 % records
1020 % records
1040 % records
1060 % records
1080 % records


In [0]:
train_df_11 = pd.DataFrame([])
for i in range(550000,600000):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_11 = train_df_11.append(rows)
    if i%10000==0:
        print(int(i*100/50000), '% records')

1100 % records
1120 % records
1140 % records
1160 % records
1180 % records


In [0]:
train_df_12 = pd.DataFrame([])
for i in range(600000,len(train)):
    rows = combined_new.loc[[train.building_id[i]]]
    train_df_12 = train_df_12.append(rows)

In [0]:
train_combined = pd.concat([train_df, train_df_1, train_df_2, train_df_3, train_df_4, train_df_5, train_df_6, train_df_7, train_df_8, train_df_9, train_df_10, train_df_11, train_df_12],axis=0)

In [0]:
train_combined.shape

(631761, 75)

In [0]:
train.shape

(631761, 14)

In [0]:
train_combined = train_combined.reset_index()
train_combined.to_csv('drive/Colab Notebooks/Building/combined_train.csv')

In [0]:
train_combined.head()

,building_id,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,...,Damaged-Repaired and used,Damaged-Rubble Clear-New building built,Damaged-Rubble clear,Damaged-Rubble unclear,Damaged-Used in risk,Not damaged,Institutional,Other,Private,Public
0,24385bfd2a2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,405d1bbebbf,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,351d9bc71f6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2be3a971166,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,34c7d073ea6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631761 entries, 0 to 631760
Data columns (total 17 columns):
building_id                              631761 non-null object
damage_grade                             631761 non-null object
has_geotechnical_risk                    631761 non-null float64
has_geotechnical_risk_fault_crack        631761 non-null int64
has_geotechnical_risk_flood              631761 non-null int64
has_geotechnical_risk_land_settlement    631761 non-null int64
has_geotechnical_risk_landslide          631761 non-null int64
has_geotechnical_risk_liquefaction       631761 non-null int64
has_geotechnical_risk_other              631761 non-null int64
has_geotechnical_risk_rock_fall          631761 non-null int64
has_repair_started                       598344 non-null float64
vdcmun_id                                631761 non-null int64
Both                                     631761 non-null uint8
Building removed                         631761 non-null uint8
E

In [0]:
train.head()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,0,0,0.0,2489
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,0,0,1.0,4423
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,0,0,0.0,3650
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,0,0,0.0,3016
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,0,0,0.0,3627


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631761 entries, 0 to 631760
Data columns (total 14 columns):
area_assesed                             631761 non-null object
building_id                              631761 non-null object
damage_grade                             631761 non-null object
district_id                              631761 non-null int64
has_geotechnical_risk                    631761 non-null float64
has_geotechnical_risk_fault_crack        631761 non-null int64
has_geotechnical_risk_flood              631761 non-null int64
has_geotechnical_risk_land_settlement    631761 non-null int64
has_geotechnical_risk_landslide          631761 non-null int64
has_geotechnical_risk_liquefaction       631761 non-null int64
has_geotechnical_risk_other              631761 non-null int64
has_geotechnical_risk_rock_fall          631761 non-null int64
has_repair_started                       598344 non-null float64
vdcmun_id                                631761 non-null int64


In [0]:
del train['district_id']

In [0]:
train.area_assesed.value_counts()

Both                   382992
Building removed       131346
Exterior                99030
Not able to inspect     16575
Interior                 1818
Name: area_assesed, dtype: int64

In [0]:
area = pd.get_dummies(train.area_assesed)

In [0]:
train.has_repair_started.value_counts()

0.0    409222
1.0    189122
Name: has_repair_started, dtype: int64

In [0]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = "NaN", strategy = 'most_frequent')
input_data = train.has_repair_started.values.reshape(-1,1)
train.has_repair_started = imp.fit_transform(input_data, train.has_repair_started)

In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631761 entries, 0 to 631760
Data columns (total 17 columns):
building_id                              631761 non-null object
damage_grade                             631761 non-null object
has_geotechnical_risk                    631761 non-null float64
has_geotechnical_risk_fault_crack        631761 non-null int64
has_geotechnical_risk_flood              631761 non-null int64
has_geotechnical_risk_land_settlement    631761 non-null int64
has_geotechnical_risk_landslide          631761 non-null int64
has_geotechnical_risk_liquefaction       631761 non-null int64
has_geotechnical_risk_other              631761 non-null int64
has_geotechnical_risk_rock_fall          631761 non-null int64
has_repair_started                       631761 non-null float64
vdcmun_id                                631761 non-null int64
Both                                     631761 non-null uint8
Building removed                         631761 non-null uint8
E

In [0]:
train = pd.concat([train, area], axis=1)
del train['area_assesed']

In [0]:
train.head()

,building_id,damage_grade,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id,Both,Building removed,Exterior,Interior,Not able to inspect
0,24385bfd2a2,Grade 4,0.0,0,0,0,0,0,0,0,0.0,2489,1,0,0,0,0
1,405d1bbebbf,Grade 2,0.0,0,0,0,0,0,0,0,1.0,4423,1,0,0,0,0
2,351d9bc71f6,Grade 1,0.0,0,0,0,0,0,0,0,0.0,3650,1,0,0,0,0
3,2be3a971166,Grade 5,0.0,0,0,0,0,0,0,0,0.0,3016,0,1,0,0,0
4,34c7d073ea6,Grade 3,0.0,0,0,0,0,0,0,0,0.0,3627,1,0,0,0,0


In [0]:
train_full = pd.merge(train, train_combined, on='building_id')
train_full.head()

,building_id,damage_grade,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,...,Damaged-Repaired and used,Damaged-Rubble Clear-New building built,Damaged-Rubble clear,Damaged-Rubble unclear,Damaged-Used in risk,Not damaged,Institutional,Other,Private,Public
0,24385bfd2a2,Grade 4,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,405d1bbebbf,Grade 2,0.0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,351d9bc71f6,Grade 1,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2be3a971166,Grade 5,0.0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,34c7d073ea6,Grade 3,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [0]:
train_full.shape

(631761, 92)

In [0]:
train_full.set_index('building_id', inplace=True)

In [0]:
imp_1 = Imputer(missing_values = "NaN", strategy = 'most_frequent')
input_data_1 = train_full.count_families.values.reshape(-1,1)
train_full.count_families = imp_1.fit_transform(input_data_1, train_full.count_families)

In [0]:
train_full.count_families.value_counts()

1.0     547837
0.0      50659
2.0      27771
3.0       4185
4.0        939
5.0        262
6.0         73
7.0         19
8.0          9
9.0          5
11.0         1
Name: count_families, dtype: int64

In [0]:
#train_full = train_full.astype(float)
train_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 631761 entries, 24385bfd2a2 to 293272232cf
Data columns (total 91 columns):
damage_grade                               631761 non-null object
has_geotechnical_risk                      631761 non-null float64
has_geotechnical_risk_fault_crack          631761 non-null int64
has_geotechnical_risk_flood                631761 non-null int64
has_geotechnical_risk_land_settlement      631761 non-null int64
has_geotechnical_risk_landslide            631761 non-null int64
has_geotechnical_risk_liquefaction         631761 non-null int64
has_geotechnical_risk_other                631761 non-null int64
has_geotechnical_risk_rock_fall            631761 non-null int64
has_repair_started                         631761 non-null float64
vdcmun_id                                  631761 non-null int64
Both                                       631761 non-null uint8
Building removed                           631761 non-null uint8
Exterior                     

In [0]:
train_full.isnull().sum().sum()

0

In [0]:
train_full.to_csv('drive/Colab Notebooks/Building/train_full.csv',index=False)